In [1]:
import tensorflow as tf 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices() )

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10689718292042749864
xla_global_id: -1
]


In [1]:
import os.path
#import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from pathlib import Path
from tqdm import tqdm
from time import perf_counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from IPython.display import Markdown, display

In [2]:
import numpy as np
import tensorflow as tf


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split=0.2)

train_gen = train_datagen.flow_from_directory('E:/PycharmProjects/peslab/GDSC SC/recycle_img/Training',
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='training')
val_gen  = train_datagen.flow_from_directory('E:/PycharmProjects/peslab/GDSC SC/recycle_img/Validation',
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='validation')

Found 37345 images belonging to 4 classes.
Found 2402 images belonging to 4 classes.


In [5]:
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[150, 150, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=4, activation='softmax'))

# Compiling the CNN
cnn.compile(optimizer = 'adam', 
            loss = 'categorical_crossentropy', 
            metrics = ['accuracy'])
cnn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 41472)             0         
                                                                 
 dense (Dense)               (None, 128)               

In [6]:
cnn.fit(x = train_gen, validation_data = val_gen, epochs = 10)

Epoch 1/10
1168/1168 [==============================] - 7578s 6s/step - loss: 0.7541 - accuracy: 0.6571 - val_loss: 0.9952 - val_accuracy: 0.5608
Epoch 2/10
1168/1168 [==============================] - 6663s 6s/step - loss: 0.5335 - accuracy: 0.7651 - val_loss: 1.0653 - val_accuracy: 0.5545
Epoch 3/10
1168/1168 [==============================] - 6708s 6s/step - loss: 0.3984 - accuracy: 0.8304 - val_loss: 1.1853 - val_accuracy: 0.5966
Epoch 4/10
1168/1168 [==============================] - 6735s 6s/step - loss: 0.2845 - accuracy: 0.8813 - val_loss: 1.3758 - val_accuracy: 0.6012
Epoch 5/10
1168/1168 [==============================] - 6700s 6s/step - loss: 0.2027 - accuracy: 0.9189 - val_loss: 1.7324 - val_accuracy: 0.5949
Epoch 6/10
1168/1168 [==============================] - 6712s 6s/step - loss: 0.1346 - accuracy: 0.9480 - val_loss: 2.3523 - val_accuracy: 0.5783
Epoch 7/10
1168/1168 [==============================] - 6718s 6s/step - loss: 0.0852 - accuracy: 0.9679 - val_loss: 2.8799 -

In [10]:
cnn.save('E:/PycharmProjects/peslab/GDSC SC/cnn_recycle.h5')

RESNET 18

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True):
    """Residual block."""
    if conv_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(x)
        shortcut = layers.BatchNormalization()(shortcut)
    else:
        shortcut = x

    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters, kernel_size, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.add([shortcut, x])
    x = layers.Activation('relu')(x)
    return x

def build_resnet18(input_shape=(150, 150, 3), num_classes=4):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    x = residual_block(x, 64, stride=1, conv_shortcut=False)
    x = residual_block(x, 64, stride=1, conv_shortcut=False)

    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128, stride=1, conv_shortcut=False)

    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256, stride=1, conv_shortcut=False)

    x = residual_block(x, 512, stride=2)
    x = residual_block(x, 512, stride=1, conv_shortcut=False)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# Build ResNet-18 model
resnet18 = build_resnet18(input_shape=[150, 150, 3], num_classes=4)

# Compile the model
resnet18.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
resnet18.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_2 (Conv2D)           (None, 75, 75, 64)           9472      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 75, 75, 64)           256       ['conv2d_2[0][0]']            
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 75, 75, 64)           0         ['batch_normalization[0][0

In [12]:
resnet18.fit(x = train_gen, validation_data = val_gen, epochs = 10)

Epoch 1/10
1168/1168 [==============================] - 7022s 6s/step - loss: 0.7515 - accuracy: 0.6538 - val_loss: 1.3177 - val_accuracy: 0.5462
Epoch 2/10
1168/1168 [==============================] - 6873s 6s/step - loss: 0.4853 - accuracy: 0.7893 - val_loss: 1.1761 - val_accuracy: 0.6249
Epoch 3/10
1168/1168 [==============================] - 6891s 6s/step - loss: 0.3658 - accuracy: 0.8486 - val_loss: 1.0656 - val_accuracy: 0.6628
Epoch 4/10
1168/1168 [==============================] - 6894s 6s/step - loss: 0.3076 - accuracy: 0.8752 - val_loss: 1.0876 - val_accuracy: 0.6478
Epoch 5/10
1168/1168 [==============================] - 6899s 6s/step - loss: 0.2450 - accuracy: 0.9017 - val_loss: 1.0956 - val_accuracy: 0.6366
Epoch 6/10
1168/1168 [==============================] - 6916s 6s/step - loss: 0.2076 - accuracy: 0.9189 - val_loss: 0.9215 - val_accuracy: 0.6790
Epoch 7/10
1168/1168 [==============================] - 6930s 6s/step - loss: 0.1659 - accuracy: 0.9364 - val_loss: 1.1404 -

In [13]:
resnet18.save('E:/PycharmProjects/peslab/GDSC SC/resnet18_recycle.h5')

Efficientnet

In [21]:
import tensorflow as tf
import tensorflow_hub as hub

# Example: Loading a ResNet model from TensorFlow Hub
model_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4'
hub_layer = hub.KerasLayer(model_url, input_shape=(150, 150, 3))

model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 2048)              23564800  
                                                                 
 dense_3 (Dense)             (None, 4)                 8196      
                                                                 
Total params: 23572996 (89.92 MB)
Trainable params: 8196 (32.02 KB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [23]:
model.fit(x = train_gen, validation_data = val_gen, epochs = 10)

Epoch 1/10
1168/1168 [==============================] - 7622s 7s/step - loss: 0.5344 - accuracy: 0.7673 - val_loss: 0.7273 - val_accuracy: 0.7027
Epoch 2/10
1168/1168 [==============================] - 6926s 6s/step - loss: 0.4274 - accuracy: 0.8132 - val_loss: 0.6647 - val_accuracy: 0.7277
Epoch 3/10
1168/1168 [==============================] - 6866s 6s/step - loss: 0.3985 - accuracy: 0.8244 - val_loss: 0.6987 - val_accuracy: 0.7107
Epoch 4/10
1168/1168 [==============================] - 6887s 6s/step - loss: 0.3946 - accuracy: 0.8289 - val_loss: 0.5772 - val_accuracy: 0.7868
Epoch 5/10
1168/1168 [==============================] - 6862s 6s/step - loss: 0.3839 - accuracy: 0.8313 - val_loss: 0.9111 - val_accuracy: 0.6919
Epoch 6/10
1168/1168 [==============================] - 6866s 6s/step - loss: 0.3914 - accuracy: 0.8306 - val_loss: 0.7202 - val_accuracy: 0.7448
Epoch 7/10
1168/1168 [==============================] - 6879s 6s/step - loss: 0.3832 - accuracy: 0.8341 - val_loss: 0.7931 -

In [24]:
model.save('E:/PycharmProjects/peslab/GDSC SC/efficientnet_recycle.h5')

In [28]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models

def build_model(num_classes):
    # Load a pre-trained ResNet model as the base
    base_model_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
    base_model = hub.KerasLayer(base_model_url, input_shape=(224, 224, 3), trainable=False)

    # Build the model
    model = models.Sequential([
        base_model,
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Set the number of classes
num_classes = 4

# Build and compile the model
model = build_model(num_classes)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_4 (KerasLayer)  (None, 2048)              23564800  
                                                                 
 dense_6 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 24615940 (93.90 MB)
Trainable params: 1051140 (4.01 MB)
Non-trainable params: 23564800 (89.89 MB)
_________________________________________________________________


In [29]:
model.fit(x = train_gen, validation_data = val_gen, epochs = 7)

Epoch 1/7
1168/1168 [==============================] - 6900s 6s/step - loss: 0.5038 - accuracy: 0.7761 - val_loss: 0.5650 - val_accuracy: 0.7448
Epoch 2/7
1168/1168 [==============================] - 6930s 6s/step - loss: 0.3835 - accuracy: 0.8266 - val_loss: 0.6720 - val_accuracy: 0.6982
Epoch 3/7
1168/1168 [==============================] - 6897s 6s/step - loss: 0.3372 - accuracy: 0.8496 - val_loss: 0.5798 - val_accuracy: 0.7748
Epoch 4/7
1168/1168 [==============================] - 6906s 6s/step - loss: 0.3012 - accuracy: 0.8690 - val_loss: 0.6143 - val_accuracy: 0.8172
Epoch 5/7
1168/1168 [==============================] - 6886s 6s/step - loss: 0.2711 - accuracy: 0.8845 - val_loss: 0.7186 - val_accuracy: 0.7619
Epoch 6/7
1168/1168 [==============================] - 6834s 6s/step - loss: 0.2483 - accuracy: 0.8955 - val_loss: 0.8158 - val_accuracy: 0.7319
Epoch 7/7
1168/1168 [==============================] - 6808s 6s/step - loss: 0.2215 - accuracy: 0.9073 - val_loss: 0.6972 - val_ac

In [30]:
model.save('E:/PycharmProjects/peslab/GDSC SC/resnet18_epoch7.h5')

In [7]:
"""def create_gen():
    # 생성기 및 데이터 증강으로 이미지 로드
    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        validation_split=0.1
    )

    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    )

    train_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath', # 파일위치 열이름
        y_col='Label', # 클래스 열이름
        target_size=(224, 224), # 이미지 사이즈
        color_mode='rgb', # 이미지 채널수
        class_mode='categorical', # Y값(Label값)
        batch_size=32,
        shuffle=True, # 데이터를 섞을지 여부
        seed=0,
        subset='training', # train 인지 val인지 설정
        rotation_range=30, # 회전제한 각도 30도
        zoom_range=0.15, # 확대 축소 15%
        width_shift_range=0.2, # 좌우이동 20%
        height_shift_range=0.2, # 상하이동 20%
        shear_range=0.15, # 반시계방햐의 각도
        horizontal_flip=True, # 좌우 반전 True
        fill_mode="nearest"
        # 이미지 변경시 보완 방법 (constant, nearest, reflect, wrap) 4개 존재
    )

    val_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=True,
        seed=0,
        subset='validation',
        rotation_range=30,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    
    return train_generator,test_generator,train_images,val_images,test_images"""

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_gen():
    # Initialize ImageDataGenerator for training with data augmentation and rescaling
    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input, # Preprocessing function
        rescale=1./255, # Rescaling factor
        validation_split=0.2, # Splitting data for validation
        rotation_range=30, # Random rotation
        zoom_range=0.15, # Random zoom
        width_shift_range=0.2, # Random width shift
        height_shift_range=0.2, # Random height shift
        shear_range=0.15, # Shearing
        horizontal_flip=True, # Horizontal flip
        fill_mode="nearest" # Fill mode for newly created pixels
    )

    # Setup training generator
    train_gen = train_datagen.flow_from_directory(
        'E:/PycharmProjects/peslab/GDSC SC/recycle_img/Training', # Training directory path
        target_size=(224, 224), # Target image size
        batch_size=32, # Batch size
        class_mode='categorical', # Class mode
        shuffle=True, # Shuffle data
        subset='training' # Subset for training
    )

    # Setup validation generator
    val_gen = train_datagen.flow_from_directory(
        'E:/PycharmProjects/peslab/GDSC SC/recycle_img/Validation', # Validation directory path
        target_size=(224, 224), # Target image size
        batch_size=32, # Batch size
        class_mode='categorical', # Class mode
        shuffle=True, # Shuffle data
        subset='validation' # Subset for validation
    )

    return train_gen, val_gen

#train_gen, val_gen = create_gen()


In [4]:
models = {
    "DenseNet121": {"model":tf.keras.applications.DenseNet121, "perf":0},
    "MobileNetV2": {"model":tf.keras.applications.MobileNetV2, "perf":0},
    "DenseNet201": {"model":tf.keras.applications.DenseNet201, "perf":0},
    "EfficientNetB0": {"model":tf.keras.applications.EfficientNetB0, "perf":0},
    "EfficientNetB1": {"model":tf.keras.applications.EfficientNetB1, "perf":0},
    "InceptionV3": {"model":tf.keras.applications.InceptionV3, "perf":0},
    "MobileNetV2": {"model":tf.keras.applications.MobileNetV2, "perf":0},
    "MobileNetV3Large": {"model":tf.keras.applications.MobileNetV3Large, "perf":0},
    "ResNet152V2": {"model":tf.keras.applications.ResNet152V2, "perf":0},
    "ResNet50": {"model":tf.keras.applications.ResNet50, "perf":0},
    "ResNet50V2": {"model":tf.keras.applications.ResNet50V2, "perf":0},
    "VGG19": {"model":tf.keras.applications.VGG19, "perf":0},
    "VGG16": {"model":tf.keras.applications.VGG16, "perf":0},
    "Xception": {"model":tf.keras.applications.Xception, "perf":0}
}
# Create the generators
#train_generator,test_generator,train_images,val_images,test_images=create_gen()
train_gen, val_gen = create_gen()
print('\n')

def get_model(model):
# Load the pretained model
    kwargs =    {'input_shape':(224, 224, 3),
                'include_top':False,
                'weights':'imagenet',
                'pooling':'avg'}
    
    pretrained_model = model(**kwargs)
    pretrained_model.trainable = False # 레이어를 동결 시켜서 훈련중 손실을 최소화 한다.
    
    inputs = pretrained_model.input

    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
    # 라벨 개수가 4개이기 때문에 Dencs도 4로 설정
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Train모델 학습
for name, model in models.items():
    
    # 전이 학습 모델 가져오기
    m = get_model(model['model'])
    models[name]['model'] = m
    
    start = perf_counter()
    
    # 모델 학습
    history = m.fit(train_gen,validation_data=val_gen,epochs=1,verbose=1)
    
    # 학습시간과 val_accuracy 저장
    duration = perf_counter() - start
    duration = round(duration,2)
    models[name]['perf'] = duration
    print(f"{name:20} trained in {duration} sec")
    
    val_acc = history.history['val_accuracy']
    models[name]['val_acc'] = [round(v,4) for v in val_acc]

Found 37345 images belonging to 4 classes.
Found 2402 images belonging to 4 classes.


1168/1168 [==============================] - 7684s 7s/step - loss: 1.0346 - accuracy: 0.5144 - val_loss: 1.4166 - val_accuracy: 0.4967
DenseNet121          trained in 7690.95 sec
1168/1168 [==============================] - 7670s 7s/step - loss: 1.0391 - accuracy: 0.5110 - val_loss: 1.5059 - val_accuracy: 0.4967
MobileNetV2          trained in 7673.25 sec
1168/1168 [==============================] - 10346s 9s/step - loss: 1.0254 - accuracy: 0.5152 - val_loss: 1.4879 - val_accuracy: 0.4967
DenseNet201          trained in 10351.04 sec
1168/1168 [==============================] - 7063s 6s/step - loss: 1.0443 - accuracy: 0.5145 - val_loss: 1.4133 - val_accuracy: 0.4967
EfficientNetB0       trained in 7067.6 sec
1168/1168 [==============================] - 7067s 6s/step - loss: 1.0438 - accuracy: 0.5143 - val_loss: 1.5829 - val_accuracy: 0.4967
EfficientNetB1       trained in 7070.47 sec
1168/1168 [======


KeyboardInterrupt



In [ ]:
# test데이터로 모델 성능 예측
for name, model in models.items():
    
    # Predict the label of the test_images
    pred = models[name]['model'].predict(test_images)
    pred = np.argmax(pred,axis=1)

    # Map the label
    labels = (train_images.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    pred = [labels[k] for k in pred]

    y_test = list(test_df.Label)
    acc = accuracy_score(y_test,pred)
    models[name]['acc'] = round(acc,4)
    print(f'**{name} has a {acc * 100:.2f}% accuracy on the test set**')
   
# Create a DataFrame with the results
models_result = []

for name, v in models.items():
    models_result.append([ name, models[name]['val_acc'][-1], 
                          models[name]['acc'],
                          models[name]['perf']])
    
df_results = pd.DataFrame(models_result, 
                          columns = ['model','val_accuracy','accuracy','Training time (sec)'])
df_results.sort_values(by='accuracy', ascending=False, inplace=True)
df_results.reset_index(inplace=True,drop=True)
df_results

In [ ]:
plt.figure(figsize = (15,5))
sns.barplot(x = 'model', y = 'accuracy', data = df_results)
plt.title('Accuracy on the test set (after 1 epoch))', fontsize = 15)
plt.ylim(0,1)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize = (15,5))
sns.barplot(x = 'model', y = 'Training time (sec)', data = df_results)
plt.title('Training time for each model in sec', fontsize = 15)
# plt.ylim(0,20)
plt.xticks(rotation=90)
plt.show()

In [ ]:
train_df,test_df = train_test_split(df, test_size=0.1, random_state=0)
train_generator,test_generator,train_images,val_images,test_images=create_gen()

model = get_model(tf.keras.applications.DenseNet201)
history = model.fit(train_images,validation_data=val_images,epochs=7)

In [ ]:
pd.DataFrame(history.history)[['accuracy','val_accuracy']].plot()
plt.title("Accuracy")
plt.show()

In [ ]:
pd.DataFrame(history.history)[['loss','val_loss']].plot()
plt.title("Loss")
plt.show()

In [ ]:
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]
    
y_test = list(test_df.Label)
acc = accuracy_score(y_test,pred)
print(f'Accuracy on the test set: {acc * 100:.2f}%')

In [ ]:
train_df,test_df = train_test_split(df, test_size=0.1, random_state=0)
train_generator,test_generator,train_images,val_images,test_images=create_gen()

model = get_model(tf.keras.applications.ResNet152V2)
history = model.fit(train_images,validation_data=val_images,epochs=5)

In [ ]:
pd.DataFrame(history.history)[['accuracy','val_accuracy']].plot()
plt.title("Accuracy")
plt.show()

In [ ]:
pd.DataFrame(history.history)[['loss','val_loss']].plot()
plt.title("Loss")
plt.show()

In [ ]:
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))
    
y_test = list(test_df.Label)
acc = accuracy_score(y_test,pred)
printmd(f'# Accuracy on the test set: {acc * 100:.2f}%')

In [ ]:
class_report = classification_report(y_test, pred, zero_division=1)
print(class_report)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cf_matrix = confusion_matrix(y_test, pred, normalize='true')
plt.figure(figsize = (10,7))
sns.heatmap(cf_matrix, annot=False, xticklabels = sorted(set(y_test)), yticklabels = sorted(set(y_test)),cbar=False)
plt.title('Normalized Confusion Matrix', fontsize = 23)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
# from PIL import Image
import pandas as pd
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))
class_dictionary = {'airplane': 0,
                    'car': 1,
                    'cat': 2,
                    'dog': 3,
                    'flower': 4,
                    'fruit': 5,
                    'motorbike': 6,
                    'person': 7}
IMAGE_SIZE    = (224, 224)
number_1 = int(input("번호를 입력하세요 : ")) # 10, 50, 100
test_image = image.load_img(test_df.iloc[number_1, 0]
                            ,target_size =IMAGE_SIZE )
test_image = image.img_to_array(test_image)
plt.imshow(test_image/255.);

test_image = test_image.reshape((1, test_image.shape[0], test_image.shape[1], test_image.shape[2]))
test_image = preprocess_input(test_image)
prediction = model.predict(test_image)

df = pd.DataFrame({'pred':prediction[0]})
df = df.sort_values(by='pred', ascending=False, na_position='first')
printmd(f"## 예측률 : {(df.iloc[0]['pred'])* 100:.2f}%")

for x in class_dictionary:
  if class_dictionary[x] == (df[df == df.iloc[0]].index[0]):
    printmd(f"### Class prediction = {x}")
    break

In [ ]:
# Display picture of the dataset with their labels
fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(20, 12),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_df.Filepath.iloc[i]))
    ax.set_title(f"True: {test_df.Label.iloc[i].split('_')[0]}\nPredicted: {pred[i].split('_')[0]}", fontsize = 15)
plt.tight_layout()
plt.show()

이미지 한장을 가지고 성능 테스트

In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import load_model


# 1. 모델 불러오기
model = load_model('E:/PycharmProjects/peslab/GDSC SC/resnet18_epoch7.h5', custom_objects={'KerasLayer': hub.KerasLayer})

# 2. 이미지 전처리
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size) # 이미지 로드 및 크기 조정
    img_array = image.img_to_array(img) # 이미지를 numpy 배열로 변환
    img_array_expanded_dims = np.expand_dims(img_array, axis=0) # 차원 확장
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims) # 모델에 맞게 전처리

# 3. 이미지를 모델에 입력하고 예측 수행
img_path = 'C:/Users/20161/Downloads/건전지.jpg' 
preprocessed_image = preprocess_image(img_path)
predictions = model.predict(preprocessed_image)

# 4. 예측 결과 출력 (여기서는 가장 높은 확률을 가진 클래스를 출력합니다)
predicted_class = np.argmax(predictions, axis=1)
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 368ms/step
Predicted class: [0]


In [14]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import load_model


# 1. 모델 불러오기
model = load_model('E:/PycharmProjects/peslab/GDSC SC/resnet18_epoch7.h5', custom_objects={'KerasLayer': hub.KerasLayer})

# 2. 이미지 전처리
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size) # 이미지 로드 및 크기 조정
    img_array = image.img_to_array(img) # 이미지를 numpy 배열로 변환
    img_array_expanded_dims = np.expand_dims(img_array, axis=0) # 차원 확장
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims) # 모델에 맞게 전처리

# 3. 이미지를 모델에 입력하고 예측 수행
img_path = 'C:/Users/20161/Downloads/스티로폼1.jpg' 
preprocessed_image = preprocess_image(img_path)
predictions = model.predict(preprocessed_image)

# 4. 예측 결과 출력 (여기서는 가장 높은 확률을 가진 클래스를 출력합니다)
predicted_class = np.argmax(predictions, axis=1)
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 370ms/step
Predicted class: [1]


In [16]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import load_model


# 1. 모델 불러오기
model = load_model('E:/PycharmProjects/peslab/GDSC SC/resnet18_epoch7.h5', custom_objects={'KerasLayer': hub.KerasLayer})

# 2. 이미지 전처리
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size) # 이미지 로드 및 크기 조정
    img_array = image.img_to_array(img) # 이미지를 numpy 배열로 변환
    img_array_expanded_dims = np.expand_dims(img_array, axis=0) # 차원 확장
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims) # 모델에 맞게 전처리

# 3. 이미지를 모델에 입력하고 예측 수행
img_path = 'C:/Users/20161/Downloads/유리병1.jpg' 
preprocessed_image = preprocess_image(img_path)
predictions = model.predict(preprocessed_image)

# 4. 예측 결과 출력 (여기서는 가장 높은 확률을 가진 클래스를 출력합니다)
predicted_class = np.argmax(predictions, axis=1)
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 367ms/step
Predicted class: [1]


In [17]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import load_model%


# 1. 모델 불러오기
model = load_model('E:/PycharmProjects/peslab/GDSC SC/resnet18_epoch7.h5', custom_objects={'KerasLayer': hub.KerasLayer})

# 2. 이미지 전처리
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size) # 이미지 로드 및 크기 조정
    img_array = image.img_to_array(img) # 이미지를 numpy 배열로 변환
    img_array_expanded_dims = np.expand_dims(img_array, axis=0) # 차원 확장
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims) # 모델에 맞게 전처리

# 3. 이미지를 모델에 입력하고 예측 수행
img_path = 'C:/Users/20161/Downloads/페트병.jpg' 
preprocessed_image = preprocess_image(img_path)
predictions = model.predict(preprocessed_image)

# 4. 예측 결과 출력 (여기서는 가장 높은 확률을 가진 클래스를 출력합니다)
predicted_class = np.argmax(predictions, axis=1)
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 369ms/step
Predicted class: [3]
